In [1]:
# 패키지
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    ElectraTokenizer,
    ElectraForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
from transformers import EarlyStoppingCallback

c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel(
    "./Data/train/감성대화말뭉치(최종데이터)_Training.xlsx", engine="openpyxl"
)

data

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51625,51626,노년,남성,재정,만성질환 무,분노,성가신,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,51627,노년,여성,재정,만성질환 무,불안,초조한,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,51628,노년,여성,재정,만성질환 무,상처,희생된,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,51629,노년,여성,대인관계,만성질환 무,불안,걱정스러운,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [11]:
data.isnull().sum()

Unnamed: 0       0
연령               0
성별               0
상황키워드            0
신체질환             0
감정_대분류           0
감정_소분류           0
사람문장1            0
시스템문장1           0
사람문장2            0
시스템문장2           0
사람문장3         8935
시스템문장3        8935
dtype: int64

In [12]:
data.duplicated().sum()

0

In [13]:
data["감정_대분류"].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [14]:
data["감정_소분류"].unique()

array(['노여워하는', '느긋', '걱정스러운', '당혹스러운', '당황', '마비된', '만족스러운', '배신당한',
       '버려진', '부끄러운', '분노', '불안', '비통한', '상처', '성가신', '스트레스 받는', '슬픔',
       '신뢰하는', '신이 난', '실망한', '악의적인', '안달하는', '안도', '억울한', '열등감', '염세적인',
       '외로운', '우울한', '고립된', '좌절한', '후회되는', '혐오스러운', '한심한', '자신하는', '기쁨',
       '툴툴대는', '남의 시선을 의식하는', '회의적인', '죄책감의', '혼란스러운', '초조한', '흥분',
       '충격 받은', '취약한', '편안한', '방어적인', '질투하는', '두려운', '눈물이 나는', '짜증내는',
       '조심스러운', '낙담한', '환멸을 느끼는', '희생된', '감사하는', '구역질 나는', '괴로워하는',
       '가난한, 불우한'], dtype=object)

# preprocessing

In [15]:
sampled_data = data.sample(frac=0.6, random_state=42)

In [16]:
sampled_data["감정_라벨"] = sampled_data["감정_대분류"].astype("category").cat.codes

X = sampled_data["사람문장1"]
y = sampled_data["감정_라벨"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# model train

In [17]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator", num_labels=6
)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [19]:
train_dataset = Dataset.from_pandas(pd.DataFrame({"text": X_train, "label": y_train}))
test_dataset = Dataset.from_pandas(pd.DataFrame({"text": X_test, "label": y_test}))

In [20]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 6196/6196 [00:01<00:00, 4249.93 examples/s]


In [21]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     warmup_steps=500,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     logging_dir='./logs',
#     fp16=True  # Mixed precision training을 활성화
# )

In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
)

c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 모델 학습
trainer.train()

  0%|          | 0/1550 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
# trainer.evaluate()

# # 모델 저장
# model.save_pretrained("./emotion_detection_model")
# tokenizer.save_pretrained("./emotion_detection_model")

100%|██████████| 97/97 [02:35<00:00,  1.60s/it]


('./emotion_detection_model\\tokenizer_config.json',
 './emotion_detection_model\\special_tokens_map.json',
 './emotion_detection_model\\vocab.txt',
 './emotion_detection_model\\added_tokens.json')

# 전체 데이터 학습 코드(시간 관계상 진행X)

In [6]:
# import pandas as pd
# from transformers import ElectraTokenizer, ElectraForSequenceClassification
# from torch.utils.data import DataLoader, Dataset
# import torch
# from tqdm import tqdm

# # 필요한 열만 추출 (사람 대화와 감정 라벨)
# conversations = data[["사람문장1", "사람문장2", "사람문장3", "감정_대분류"]]

# # NaN 값 제거
# conversations = conversations.dropna(subset=["사람문장1", "감정_대분류"])

# # 감정 라벨 필터링 (기쁨, 분노, 슬픔, 상처)
# valid_emotions = ["기쁨", "분노", "슬픔", "상처"]
# conversations = conversations[conversations["감정_대분류"].isin(valid_emotions)]

# # 감정 라벨 매핑 딕셔너리 생성 (기쁨, 분노, 슬픔, 상처만)
# emotion_labels = {emotion: idx for idx, emotion in enumerate(valid_emotions)}
# print(f"Emotion labels: {emotion_labels}")

# # 라벨 매핑
# conversations["label"] = conversations["감정_대분류"].map(emotion_labels)

# # 라벨을 정수형으로 변환
# conversations["label"] = conversations["label"].astype(int)


# # 데이터셋 구성
# class DialogueDataset(Dataset):
#     def __init__(self, conversations, tokenizer, max_length=512):
#         self.conversations = conversations
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.conversations)

#     def __getitem__(self, idx):
#         conversation = self.conversations.iloc[idx]
#         dialogue_text = f"{conversation['사람문장1']} {conversation['사람문장2'] or ''} {conversation['사람문장3'] or ''}"
#         label = conversation["label"]

#         # 대화 텍스트 토크나이징
#         encoding = self.tokenizer(
#             dialogue_text.strip(),
#             max_length=self.max_length,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#         )

#         return {
#             "input_ids": encoding["input_ids"].squeeze(),
#             "attention_mask": encoding["attention_mask"].squeeze(),
#             "label": torch.tensor(label, dtype=torch.long),
#         }


# # KoELECTRA 모델 및 토크나이저 로드
# tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# num_labels = len(emotion_labels)
# model = ElectraForSequenceClassification.from_pretrained(
#     "monologg/koelectra-base-v3-discriminator", num_labels=num_labels
# )

# # 데이터셋 및 DataLoader 구성
# dataset = DialogueDataset(conversations, tokenizer)
# dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


# # 학습 함수 정의 (tqdm 추가)
# def train(model, dataloader, optimizer, num_epochs=3):
#     model.train()
#     for epoch in range(num_epochs):
#         total_loss = 0
#         # tqdm을 사용하여 진행 상황을 표시
#         progress_bar = tqdm(
#             dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"
#         )

#         for batch in progress_bar:
#             optimizer.zero_grad()

#             input_ids = batch["input_ids"]
#             attention_mask = batch["attention_mask"]
#             labels = batch["label"]

#             # 모델 예측 및 loss 계산
#             outputs = model(
#                 input_ids=input_ids, attention_mask=attention_mask, labels=labels
#             )
#             loss = outputs.loss

#             # 역전파 및 최적화
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#             # tqdm 진행바에 현재 평균 loss 추가
#             progress_bar.set_postfix({"loss": total_loss / len(progress_bar)})

#         avg_loss = total_loss / len(dataloader)
#         print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")


# # Optimizer 설정
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# # 모델 학습
# train(model, dataloader, optimizer, num_epochs=3)

Emotion labels: {'기쁨': 0, '분노': 1, '슬픔': 2, '상처': 3}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3:  11%|█         | 903/8389 [58:06<8:01:46,  3.86s/batch, loss=0.119]  


KeyboardInterrupt: 

In [ ]:
# # 모델 저장
# model.save_pretrained("./trained_emotion_model")
# tokenizer.save_pretrained("./trained_emotion_model")